## Problem context and explanation.

##### As COVID-19 is spreading throug the world the enconomic impact on develop countries has been desvatating. Bogota is the capital city of Colombia and the city i currently live, as a capital city the number of cases are increasing and identify high risk places is crucial.

##### The following project tries to identify the locaties in Bogota wich migh encountered high risk, usign the amount of places located in the each one of them and clustering it using GOV public cases of confirmed COVID-19 cases.

##### This information migth be very usefull as localities with similar cases migth be warn tryingto avoid fatalities as possibly

## Data to be used.

##### Foursquare API will be used to determined all major places of interest for each locality in Bogota, this´ll allow us to idetntify if supermarkets, coffee, bars or any major placeof interest are similar betweer them

##### Bogota COVID Data Frame is a recopilation of confirmed COVID-19 cases, it includes a identifier, a date and city of case (for all it´s Bogota as the Data Frame shows only Bogota´s city data) the date of confirmation,the localitie, age and sex of the case. it also includes the case status and the location of where´s being handled

## Methodology and developmet

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                       

##### importing Bogota Data Frame

In [2]:
!wget -q -O 'bogota_df' https://datosabiertos.bogota.gov.co/dataset/44eacdb7-a535-45ed-be03-16dbbea6f6da/resource/b64ba3c4-9e41-41b8-b3fd-2da21d627558/download/osb_enftransm-covid-19.csv
print('Data downloaded!')

Data downloaded!


In [3]:
pd_bogota= pd.read_csv('bogota_df',encoding = "ISO-8859-1")
pd_bogota.head()

,Fecha de diagnóstico,Ciudad de residencia,Localidad de residencia,Edad,Sexo,Tipo de caso,Ubicación,Estado
0,6/03/2020,Bogotá,Usaquén,19,F,Importado,Casa,Recuperado
1,10/03/2020,Bogotá,Engativá,22,F,Importado,Casa,Recuperado
2,10/03/2020,Bogotá,Engativá,28,F,Importado,Casa,Recuperado
3,12/03/2020,Bogotá,Fontibón,36,F,Importado,Casa,Recuperado
4,12/03/2020,Bogotá,Kennedy,42,F,Importado,Casa,Recuperado


In [4]:
pd_bogota.drop(['Fecha de diagnóstico','Tipo de caso','Ciudad de residencia','Ubicación'],axis=1, inplace=True)
pd_bogota.head()

,Localidad de residencia,Edad,Sexo,Estado
0,Usaquén,19,F,Recuperado
1,Engativá,22,F,Recuperado
2,Engativá,28,F,Recuperado
3,Fontibón,36,F,Recuperado
4,Kennedy,42,F,Recuperado


In [6]:
pd_bogota.sort_values(["Localidad de residencia"],axis=0,ascending=True,inplace=True) 
pd_bogota.head()

,Localidad de residencia,Edad,Sexo,Estado
21628,Antonio Nariño,15,F,Recuperado
48844,Antonio Nariño,64,F,Leve
7548,Antonio Nariño,25,F,Recuperado
34317,Antonio Nariño,91,F,Leve
7547,Antonio Nariño,39,M,Recuperado


### obtaining each locality location.

In [38]:
index = pd.Index(pd_bogota['Localidad de residencia'])
localidades=index.value_counts()
pd_localides=pd.DataFrame(localidades)
pd_localides.head()

,Localidad de residencia
Kennedy,9215
Sin Dato,6838
Suba,5816
Bosa,5354
Ciudad Bolívar,4705


In [68]:
coorde=[]
localidades=(pd_localides.index.values)
for localidades in localidades:
    url= 'http://open.mapquestapi.com/geocoding/v1/address?key=	ZAzqtn4ozsspPAhGrUl8LyjUYyBavwaO&location={},Bogota,Colombia'.format(localidades)
    results = requests.get(url).json()
    latlong=results["results"][0]['locations'][0]['latLng']
    lat=latlong['lat']
    lon=latlong['lng']
    coorde.append([localidades,lat,lon])
coorde

[['Kennedy', 4.629451, -74.149927],
 ['Sin Dato', 39.78373, -100.445882],
 ['Suba', 4.761197, -74.082518],
 ['Bosa', 4.625515, -74.20032],
 ['Ciudad Bolívar', 4.537495, -74.152737],
 ['Engativá', 4.696628, -74.10619],
 ['San Cristóbal', 4.548658, -74.047473],
 ['Rafael Uribe Uribe', 4.57349, -74.119207],
 ['Usaquén', 4.695047, -74.031493],
 ['Fontibón', 4.678737, -74.146988],
 ['Usme', 4.470934, -74.125357],
 ['Puente Aranda', 4.619437, -74.106645],
 ['Tunjuelito', 4.560354, -74.128931],
 ['Santa Fe', 4.594803, -74.033742],
 ['Fuera de Bogotá', 39.78373, -100.445882],
 ['Los Mártires', 4.608375, -74.086538],
 ['Antonio Nariño', 4.588253, -74.097455],
 ['Chapinero', 4.649911, -74.046465],
 ['Teusaquillo', 4.641244, -74.086337],
 ['Barrios Unidos', 4.67025, -74.075196],
 ['La Candelaria', 4.596515, -74.073492]]

In [72]:
pd_adress=pd.DataFrame(coorde)
names=['Localidad de residencia','lat','lon']
pd_adress.columns= names
pd_adress

,Localidad de residencia,lat,lon
0,Kennedy,4.629451,-74.149927
1,Sin Dato,39.783730,-100.445882
2,Suba,4.761197,-74.082518
3,Bosa,4.625515,-74.200320
4,Ciudad Bolívar,4.537495,-74.152737
5,Engativá,4.696628,-74.106190
6,San Cristóbal,4.548658,-74.047473
7,Rafael Uribe Uribe,4.573490,-74.119207
8,Usaquén,4.695047,-74.031493
9,Fontibón,4.678737,-74.146988


In [87]:
#pd_localides.reset_index(inplace=True)
#pd_localides.columns=['Localidad de residencia','Conteo']
pb_analisis= pd.merge(pd_localides, pd_adress,on=['Localidad de residencia'])
pb_analisis.drop(['level_0','index'],axis=1, inplace=True)
pb_analisis.drop([1,14],axis=0, inplace=True)
pb_analisis

,Localidad de residencia,Conteo,lat,lon
0,Kennedy,9215,4.629451,-74.149927
2,Suba,5816,4.761197,-74.082518
3,Bosa,5354,4.625515,-74.200320
4,Ciudad Bolívar,4705,4.537495,-74.152737
5,Engativá,4079,4.696628,-74.106190
6,San Cristóbal,2780,4.548658,-74.047473
7,Rafael Uribe Uribe,2624,4.573490,-74.119207
8,Usaquén,2433,4.695047,-74.031493
9,Fontibón,2157,4.678737,-74.146988
10,Usme,2112,4.470934,-74.125357


### Now we have our Data frame ready for analisys, in the next few lines we'll be getting the spots from each locality and onehot encoding them.

#### we´ll be usig the lab funtion to get the info.

In [109]:
CLIENT_ID = 'EAHE03PNFZIQ2QGKN4DCZRPCMTNC531BIT3WED2N0FVXFEPN' # your Foursquare ID
CLIENT_SECRET = 'HRDH3HI04KD2OU34BFAEMGZC23WFQDC0Y0BQZ0PKAHWXUSWS' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # limit of number of venues returned by Foursquare API
radius = 2000 # define radius
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [110]:
Bogota_venues= getNearbyVenues(names=pb_analisis['Localidad de residencia'],latitudes=pb_analisis['lat'],longitudes=pb_analisis['lon'])

Kennedy
Suba
Bosa
Ciudad Bolívar
Engativá
San Cristóbal
Rafael Uribe Uribe
Usaquén
Fontibón
Usme
Puente Aranda
Tunjuelito
Santa Fe
Los Mártires
Antonio Nariño
Chapinero
Teusaquillo
Barrios Unidos
La Candelaria


In [111]:
Bogota_onehot = pd.get_dummies(Bogota_venues[['Venue Category']], prefix="", prefix_sep="")
Bogota_onehot['Neighborhood'] = Bogota_venues['Neighborhood'] 
fixed_columns = [Bogota_onehot.columns[-1]] + list(Bogota_onehot.columns[:-1])
Bogota_onehot = Bogota_onehot[fixed_columns]

In [113]:
Bogota_grouped = Bogota_onehot.groupby('Neighborhood').mean().reset_index()
Bogota_grouped

,Neighborhood,Yoga Studio,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Bed & Breakfast,Big Box Store,Bike Rental / Bike Share,Bistro,Bookstore,Bowling Alley,Bowling Green,Brazilian Restaurant,Breakfast Spot,Brewery,Building,Burger Joint,Burrito Place,Bus Stop,Business Service,Café,Cajun / Creole Restaurant,Car Wash,Caribbean Restaurant,Casino,Castle,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Bookstore,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Convenience Store,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Farmers Market,Fast Food Restaurant,Fish Market,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden Center,Gastropub,General Entertainment,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Hawaiian Restaurant,Historic Site,History Museum,Hockey Field,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Latin American Restaurant,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,National Park,Nature Preserve,New American Restaurant,Nightclub,Optical Shop,Outdoors & Recreation,Outlet Mall,Paintball Field,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Planetarium,Playground,Plaza,Pub,Public Art,Racetrack,Recreation Center,Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Student Center,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Theater,Theme Park,Trail,Train Station,University,Vegetarian / Vegan Restaurant,Water Park,Whisky Bar,Wine Bar,Wings Joint,Women's Store
0,Antonio Nariño,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.00000,0.000000,0.000000,0.072464,0.028986,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.014493,0.00,0.00,0.000000,0.000000,0.00000,0.043478,0.00,0.000000,0.0,0.014493,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.014493,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.028986,0.00,0.000000,0.00000,0.00,0.028986,0.057971,0.014493,0.00000,0.00,0.014493,0.014493,0.00000,0.000000,0.000000,0.00,0.00,0.000000,0.014493,0.014493,0.000000,0.014493,0.014493,0.000000,0.043478,0.014493,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.014493,0.014493,0.014493,0.000000,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.014493,0.014493,0.00,0.014493,0.000000,0.000000,0.0,0.000000,0.00,0.00000,0.00000,0.000000,0.00,0.000000,0.014493,0.043478,0.00,0.00000,0.000000,0.014493,0.014493,0.101449,0.00000,0.00,0.00000,0.000000,0.00,0.00,0.000000,0.043478,0.014493,0.043478,0.00,0.00,0.043478,0.00000,0.028986,0.014493,0.000000,0.014493,0.00000,0.014493,0.000000,0.000000,0.000000,0.0,0.00,0.014493,0.00,0.000000,0.000000,0.028986,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.00000,0.00,0.00,0.00000,0.000000,0.00
1,Barrios Unidos,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.01000,0.000000,0.00000,0.000000,0.020000,0.010000,0.050000,0.000000,0.00000,0.00,0.000000,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00000,0.050000,0.01,0.000000,0.0,0.010000,0

#### Now will make 5 clusters to compare how similar are the local hotspots beteween Bogota.

In [139]:
# set number of clusters
kclusters = 9

Bogota_clustering = Bogota_grouped.drop('Neighborhood', 1)


# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Bogota_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([1, 1, 0, 6, 4, 1, 1, 1, 6, 6], dtype=int32)

In [140]:
neighborhoods_venues_sorted= Bogota_grouped[['Neighborhood','Yoga Studio']]
neighborhoods_venues_sorted.drop(['Yoga Studio'],axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.columns=['Cluster Labels','Localidad de residencia']
Bogota_merged = pb_analisis
Bogota_merged = Bogota_merged.join(neighborhoods_venues_sorted.set_index('Localidad de residencia'), on='Localidad de residencia')
Bogota_merged.head() 

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Localidad de residencia,Conteo,lat,lon,Cluster Labels
0,Kennedy,9215,4.629451,-74.149927,1
2,Suba,5816,4.761197,-74.082518,1
3,Bosa,5354,4.625515,-74.200320,0
4,Ciudad Bolívar,4705,4.537495,-74.152737,4
5,Engativá,4079,4.696628,-74.106190,1


#### let's visualize the clusters on a map

In [142]:
latitude= 4.60971
longitude= -74.08175
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bogota_merged['lat'], Bogota_merged['lon'], Bogota_merged['Localidad de residencia'], Bogota_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### As Covid-19 propagtes throw physical contact this data allow us to wich Bogota neighborhood thepandemic migth have a similar behaviour

##### now lets cluster the locaties by the current numbers of cases

In [144]:
# set number of clusters
kclusters = 9

Bogota_clusterin_cases = pd_localides.drop('Localidad de residencia',1)


# run k-means clustering
kmeans_cases = KMeans(n_clusters=kclusters, random_state=0).fit(Bogota_clustering)

# check cluster labels generated for each row in the dataframe
kmeans_cases.labels_[0:10] 


array([1, 1, 0, 6, 4, 1, 1, 1, 6, 6], dtype=int32)

In [147]:
#Bogota_merged.insert(0, 'Cluster Labels Cases', kmeans.labels_)
Bogota_merged

,Cluster Labels Cases,Localidad de residencia,Conteo,lat,lon,Cluster Labels
0,1,Kennedy,9215,4.629451,-74.149927,1
2,1,Suba,5816,4.761197,-74.082518,1
3,0,Bosa,5354,4.625515,-74.200320,0
4,6,Ciudad Bolívar,4705,4.537495,-74.152737,4
5,4,Engativá,4079,4.696628,-74.106190,1
6,1,San Cristóbal,2780,4.548658,-74.047473,2
7,1,Rafael Uribe Uribe,2624,4.573490,-74.119207,7
8,1,Usaquén,2433,4.695047,-74.031493,6
9,6,Fontibón,2157,4.678737,-74.146988,1
10,6,Usme,2112,4.470934,-74.125357,5


In [155]:
latitude= 4.60971
longitude= -74.08175
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Bogota_merged['lat'], Bogota_merged['lon'], Bogota_merged['Localidad de residencia'], Bogota_merged['Cluster Labels Cases']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.5).add_to(map_clusters)
for lat, lon, poi, cluster in zip(Bogota_merged['lat'], Bogota_merged['lon'], Bogota_merged['Localidad de residencia'], Bogota_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow,
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=1).add_to(map_clusters)
       
map_clusters

#### So the above map is the desirable result for our case, in this we have been able to mix the localities (small cities) in Bogota and show similarities in the cases presented from COvid 19 and the places were the transmission was possible. The utility of this is that local Govmigth be able to take more focalized approaches for each locality in order to let the economy gain traction. Also if they monitor the map continouly they can